In [97]:
# !pip install ipywidgets
import joblib
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

In [98]:
# Charger le modèle depuis le fichier
model = joblib.load('model.pkl')

# Vérifiez que le modèle a été correctement chargé
print(model)

In [99]:
# Charger le DataFrame test_data depuis un fichier CSV
test_data = pd.read_csv('test_data.csv')

test_data.head()

,Arrondissement,Prix (€),Prix mensuel (€),Pièce(s),Surface (m2),Jours écoulés
0,75005,352000,10057,2,35,1963.0
1,75001,350000,8750,2,40,1652.0
2,75017,1850000,13704,5,135,2422.0
3,75001,2775000,11758,7,236,1284.0
4,75012,195000,10833,1,18,1767.0


In [100]:
# Charger la date de référence depuis un fichier texte
with open('reference_date.txt', 'r') as file:
    reference_date_str = file.read().strip()

reference_date = pd.to_datetime(reference_date_str)
print(reference_date)


2016-09-27 00:00:00


In [101]:
def plot_predictions(dateInput):
    # Calculer les jours écoulés pour la date future
    future_date = pd.to_datetime(dateInput, format='%Y-%m-%d')
    days_elapsed_future = (future_date - reference_date).days

    # Créer un nouveau DataFrame avec les valeurs modifiées pour les jours écoulés
    test_data_future = test_data.copy()
    test_data_future['Jours écoulés'] = days_elapsed_future

    # Prédire les prix pour la date future
    predictions_future = model.predict(test_data_future)

    # Exemple de valeurs réelles
    valeurs_reelles = test_data['Prix (€)']

    # Créer un DataFrame combinant les valeurs réelles et les valeurs prédites
    df_results = pd.DataFrame({
        'Index': range(len(valeurs_reelles)),
        'Valeurs Réelles': valeurs_reelles,
        'Valeurs Prédites Futur': predictions_future
    })

    # Calculer le pourcentage de hausse
    mean_valeurs_reelles = valeurs_reelles.mean()
    mean_predictions_future = predictions_future.mean()
    pourcentage_hausse = (mean_predictions_future / mean_valeurs_reelles - 1) * 100

    # Créer le graphique
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_results['Index'], y=df_results['Valeurs Réelles'],
                             mode='markers', name='Valeurs Réelles', marker=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df_results['Index'], y=df_results['Valeurs Prédites Futur'],
                             mode='markers', name='Valeurs Prédites Futur', marker=dict(color='red')))

    fig.update_layout(title='Comparaison des Valeurs Réelles et Prédites',
                      xaxis_title='Index',
                      yaxis_title='Prix (€)')

    return fig, pourcentage_hausse

def generate_dates(start_year, end_year, month=1, day=1):
    dates = [f"{year}-{month:02d}-{day:02d}" for year in range(start_year, end_year + 1)]
    return dates

def predict_price(surface, pieces, arrondissement, dateInput):
    # Calculer les jours écoulés pour la date future
    future_date = pd.to_datetime(dateInput, format='%Y-%m-%d')
    days_elapsed_future = (future_date - reference_date).days

    # Créer un DataFrame pour la prédiction
    input_data = pd.DataFrame({
        'Surface (m2)': [surface],
        'Pièce(s)': [pieces],
        'Arrondissement': [arrondissement],
        'Jours écoulés': [days_elapsed_future]
    })

    predicted_price = model.predict(input_data)[0]

    return predicted_price

In [106]:
# Initialiser l'application Dash
app = dash.Dash(__name__)

# Liste des dates disponibles pour le menu déroulant
dates = generate_dates(2025, 2070)

app.layout = html.Div([
    html.Div([
        html.H1('Prix Immobilier à Paris',
                 style={'textAlign': 'center'}
        ),
        html.P('Les prédictions sont basées sur un modèle de Machine Learning entraîné sur des données historiques.'),
        html.P('Sélectionnez une date pour voir les prédictions de prix moyens des biens immobiliers à Paris.'),
        dcc.Dropdown(
            id='date-dropdown',
            options=[{'label': date, 'value': date} for date in dates],
            value='2025-01-01',
            placeholder='Sélectionnez une date',
            style={'color': 'black', 'background-color': 'lightgrey'}
        ),
        html.Div(id='percentage-hausse',
                style={'font-size': '20px', 
                       'margin': '10px 0', 
                       'textAlign': 'center',
                       'font-weight': 'bold'}
        ),
        dcc.Graph(id='price-graph')
    ]),

    html.Div([
        html.H1('Prix Immobilier à Paris en fonction des caractéristiques du bien',
                 style={'textAlign': 'center'}
        ),
        html.Div(id='predicted-price', style={'font-size': '20px', 
                                              'margin-top': '20px', 
                                              'textAlign': 'center',
                                              'font-weight': 'bold'}
        ),
        dcc.Input(id='input-surface', type='number', placeholder='Surface (m2)', style={'margin': '10px'}),
        dcc.Input(id='input-pieces', type='number', placeholder='Nombre de pièces', style={'margin': '10px'}),
        dcc.Dropdown(
            id='dropdown-arrondissement',
            options=[{'label': str(i), 'value': i} for i in range(1, 21)],
            placeholder='Arrondissement',
            style={'margin': '10px', 'color': 'black', 'background-color': 'lightgrey'}
        ),
        dcc.Dropdown(
            id='dropdown-date',
            options=[{'label': f'{year}-01-01', 'value': f'{year}-01-01'} for year in range(2025, 2051)],
            value='2025-01-01',
            placeholder='Sélectionnez une date',
            style={'margin': '10px', 'color': 'black', 'background-color': 'lightgrey'}
        ),
        html.Button('Prédire', id='predict-button', n_clicks=0, style={'margin': '10px'})
    ])
], style={'margin': '0 auto',
          'max-width': '800px',
          'padding': '20px',
          'font-family': 'Arial, sans-serif',
          'color': 'black',
          'background-color': 'white'})

@app.callback(
    [Output('price-graph', 'figure'), Output('percentage-hausse', 'children')],
    Input('date-dropdown', 'value'),
)

def update_graph(dateInput):
    fig, pourcentage_hausse = plot_predictions(dateInput)
    return fig, f"{pourcentage_hausse:.1f}% de hausse des prix moyens"

@app.callback(
    Output('predicted-price', 'children'),
    [Input('predict-button', 'n_clicks')],
    [Input('input-surface', 'value'),
     Input('input-pieces', 'value'),
     Input('dropdown-arrondissement', 'value'),
     Input('dropdown-date', 'value')]
)
def update_prediction(n_clicks, surface, pieces, arrondissement, dateInput):
    if surface is None or pieces is None or arrondissement is None or dateInput is None:
        return "Veuillez entrer tous les champs"
    predicted_price = predict_price(surface, pieces, arrondissement, dateInput)
    formatted_price = f"{predicted_price:,.2f}".replace(",", " ").replace(".", ",")
    return f"Le prix prédit est : {formatted_price} €"

# Exécuter l'application
if __name__ == '__main__':
    app.run_server(debug=True)
